In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "cmn.txt"

In [4]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 10000
Number of unique input tokens: 72
Number of unique output tokens: 2554
Max sequence length for inputs: 28
Max sequence length for outputs: 22


In [5]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [6]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")

Epoch 1/100
125/125 [==============================] - 35s 278ms/step - loss: 2.2234 - accuracy: 0.6461 - val_loss: 2.7042 - val_accuracy: 0.5636
Epoch 2/100
125/125 [==============================] - 36s 291ms/step - loss: 1.8887 - accuracy: 0.6900 - val_loss: 2.4042 - val_accuracy: 0.6273
Epoch 3/100
125/125 [==============================] - 36s 284ms/step - loss: 1.7340 - accuracy: 0.7245 - val_loss: 2.2893 - val_accuracy: 0.6499
Epoch 4/100
125/125 [==============================] - 36s 289ms/step - loss: 1.6373 - accuracy: 0.7367 - val_loss: 2.2102 - val_accuracy: 0.6552
Epoch 5/100
125/125 [==============================] - 38s 307ms/step - loss: 1.5640 - accuracy: 0.7473 - val_loss: 2.1282 - val_accuracy: 0.6695
Epoch 6/100
125/125 [==============================] - 39s 309ms/step - loss: 1.4884 - accuracy: 0.7551 - val_loss: 2.0784 - val_accuracy: 0.6776
Epoch 7/100
125/125 [==============================] - 36s 289ms/step - loss: 1.4320 - accuracy: 0.7613 - val_loss: 2.0117 -

In [7]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [8]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: 嗨。

-
Input sentence: Hi.
Decoded sentence: 嗨。

-
Input sentence: Run.
Decoded sentence: 你用跑的。

-
Input sentence: Wait!
Decoded sentence: 等等！

-
Input sentence: Wait!
Decoded sentence: 等等！

-
Input sentence: Hello!
Decoded sentence: 你好。

-
Input sentence: I won!
Decoded sentence: 我赢了。

-
Input sentence: Oh no!
Decoded sentence: 不会吧。

-
Input sentence: Cheers!
Decoded sentence: 乾杯!

-
Input sentence: Got it?
Decoded sentence: 你懂了吗？

-
Input sentence: He ran.
Decoded sentence: 他跑了。

-
Input sentence: Hop in.
Decoded sentence: 跳进来。

-
Input sentence: I quit.
Decoded sentence: 我退出。

-
Input sentence: I'm OK.
Decoded sentence: 我沒事。

-
Input sentence: I'm up.
Decoded sentence: 我已经起来了。

-
Input sentence: Listen.
Decoded sentence: 听着。

-
Input sentence: No way!
Decoded sentence: 没门！

-
Input sentence: No way!
Decoded sentence: 没门！

-
Input sentence: Really?
Decoded sentence: 你确定？

-
Input sentence: Try it.
Decoded sentence: 试试吧。



In [18]:
from nltk.translate.bleu_score import corpus_bleu
references = [[['this', 'is', 'a', 'test'], ['this', 'is' 'test']]]
candidates = [['this', 'is', 'a', 'test']]
score = corpus_bleu(references, candidates)
print(score)

from nltk.translate.bleu_score import sentence_bleu
reference = [['this', 'is', 'a', 'test'], ['this', 'is' 'test']]
candidate = ['this', 'is', 'a', 'test']
score = sentence_bleu(reference, candidate)
print(score)

1.0
1.0


## Bleu testing

In [39]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import corpus_bleu
list_input_text=[]
list_output_text=[]
list_real_text=[]
for seq_index in range(150):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    list_real_text.append(target_texts[seq_index + 1][1:])
    list_output_text.append(decoded_sentence)
    #print("-")
    #print("Input sentence:", input_texts[seq_index])
    #print("Decoded sentence:", decoded_sentence)
print(list_real_text)
print(list_output_text)
score = 0
for i in range(50):
    score += sentence_bleu(list_real_text, list_output_text[i])
score = score/50
print("bleu score50: ",score)

for i in range(100):
    score += sentence_bleu(list_real_text, list_output_text[i])
score = score/100
print("bleu score100: ",score)

for i in range(150):
    score += sentence_bleu(list_real_text, list_output_text[i])
score = score/150
print("bleu score150: ",score)

['你好。\n', '你用跑的。\n', '等等！\n', '等一下！\n', '你好。\n', '我赢了。\n', '不会吧。\n', '乾杯!\n', '你懂了吗？\n', '他跑了。\n', '跳进来。\n', '我退出。\n', '我沒事。\n', '我已经起来了。\n', '听着。\n', '不可能！\n', '没门！\n', '你确定？\n', '试试吧。\n', '我们来试试。\n', '为什么是我？\n', '去问汤姆。\n', '好棒！\n', '冷静点。\n', '公平点。\n', '友善点。\n', '和气点。\n', '友善点。\n', '联系我。\n', '联系我们。\n', '进来。\n', '找到汤姆。\n', '滾出去！\n', '出去！\n', '走開！\n', '滾！\n', '走開！\n', '回家吧。\n', '再见！\n', '告辞！\n', '坚持。\n', '等一下！\n', '坚持。\n', '他来了。\n', '他跑。\n', '帮我一下。\n', '帮帮我们吧！\n', '去打汤姆。\n', '坚持。\n', '抱抱汤姆！\n', '请抱紧汤姆。\n', '我同意。\n', '我生病了。\n', '我很难过。\n', '我濕了。\n', '没关系。\n', '是我。\n', '来加入我们吧。\n', '留着吧。\n', '吻我。\n', '完美！\n', '再见！\n', '閉嘴！\n', '不管它。\n', '拿走吧。\n', '醒醒！\n', '去清洗一下。\n', '我们知道。\n', '欢迎。\n', '谁赢了？\n', '为什么不？\n', '你跑。\n', '算你狠。\n', '往后退点。\n', '往后退点。\n', '静静的，别动。\n', '我一无所知。\n', '把他铐上。\n', '往前开。\n', '走開！\n', '滾！\n', '趴下！\n', '滾！\n', '滚。\n', '滚。\n', '醒醒吧。\n', '做得好！\n', '干的好！\n', '抓住汤姆。\n', '抓住他。\n', '玩得開心。\n', '他来试试。\n', '多可爱啊！\n', '你就随了我的意吧。\n', '趕快!\n', '快点！\n', '快点。\n', '我做到了！\n', '我忘了。\n', '我放